In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

import numpy as np

import call_my_bluff as cmb

In [2]:
state = cmb.game.initialize_game(3)

In [3]:
state

State(num_players=3, bet_index=-1, player_curr=2, player_prev=None, turn_order=[2, 0, 1], num_dice=[5, 5, 5], dice=[[5, 1, 4, 0, 3], [1, 1, 2, 1, 0], [0, 4, 2, 4, 5]], dice_locked=[[False, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False]], action_log=[])

In [6]:
state = cmb.initialize_game(3)

count = 0
while not cmb.game_over(state):

    observation = cmb.player_observation(state)

    state = cmb.player_action(state, cmb.Action(type=cmb.ActionType.BET, bet_index=state.bet_index + 1))




    if count == 12:
        state.num_dice = [0, 0, 1]

    count += 1
    print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13


In [7]:
cmb.render(state)

Player 0 bet 1 0s
Player 1 bet 1 1s
Player 2 bet 1 2s
Player 0 bet 1 3s
Player 1 bet 1 4s
Player 2 bet 1 5s
Player 0 bet 2 0s
Player 1 bet 2 1s
Player 2 bet 2 2s
Player 0 bet 2 3s
Player 1 bet 2 4s
Player 2 bet 3 0s
Player 0 bet 3 1s


In [12]:
x = [1, 1, 2, 3]
x.remove(1)

In [13]:
x

[1, 2, 3]

In [16]:
action = cmb.ActionType.CALL
if action == cmb.ActionType.BET:
    print("call")

In [ ]:
# def play_game(num_players):
#     game_state = initialize_game(num_players)
#     history = [game_state]
#     while not game_over(game_state):
#         render(game_state)
#         player = next_player(game_state)
#         action = get_user_input(game_state, player)
#         try:
#             if action.type == "bet":
#                 game_state = make_bet(game_state, player, action.bet)
#             elif action.type == "reroll":
#                 game_state = reroll_dice(game_state, player, action.dice_to_lock)
#             elif action.type == "call":
#                 game_state = call(game_state, player)
#             history.append(game_state)
#         except InvalidActionError as e:
#             handle_invalid_action(e)
#     render(game_state)
#     return history

In [51]:
class SimpleAgent:

    def policy(self, state):
        player = state["player"]
        dice_roll = state["dice_roll"]
        locked_dice = state["locked_dice"]
        unknown_dice_per_competitor = state["unknown_dice_per_competitor"]
        known_dice_per_competitor = state["known_dice_per_competitor"]
        competitors = list(unknown_dice_per_competitor.keys())
        log = state["log"]
        bet_index = state["bet_index"]

        if bet_index is None:
            return {"call": False, "reroll": False, "bet": 0}
        
        (num_dice_bet, dice_value) = cmb.bet_index_to_bet(bet_index)
        
        total_dice = len(dice_roll) + np.sum(list(unknown_dice_per_competitor.values())) + \
            np.sum(np.stack(list(known_dice_per_competitor.values()), axis=0))
        
        if dice_value == 5:
            expected_dice = total_dice / 6
        else:
            expected_dice = total_dice / 3

        if num_dice_bet > expected_dice or bet_index == 109:
            return {"call": True}
        else:
            return {"call": False, "reroll": False, "bet": bet_index + 1}


In [91]:
class MaxAgent:

    def policy(self, state):
        player = state["player"]
        dice_roll = state["dice_roll"]
        locked_dice = state["locked_dice"]
        unknown_dice_per_competitor = state["unknown_dice_per_competitor"]
        known_dice_per_competitor = state["known_dice_per_competitor"]
        competitors = list(unknown_dice_per_competitor.keys())
        log = state["log"]
        bet_index = state["bet_index"]
                
        total_unknown_dice = np.sum(list(unknown_dice_per_competitor.values()))
        known_dice= np.zeros((6,), dtype=np.int32)
        max_bet_indices = []
        for i in range(6):
            known_dice[i] += np.sum(dice_roll == i)
            for competitor in competitors:
                known_dice[i] += np.sum(known_dice_per_competitor[competitor] == i)

            if i == 5:
                expected_num_dice = total_unknown_dice / 6.0 + known_dice[i]
            else:
                expected_num_dice = total_unknown_dice / 3.0 + known_dice[i] + known_dice[5]

            expected_num_dice = np.floor(expected_num_dice)
            if expected_num_dice == 0:
                max_bet_indices.append(0)
            else:
                max_bet_indices.append(cmb.bet_to_bet_index((expected_num_dice, i)))
        
        max_bet_index = int(np.max(max_bet_indices))
        if bet_index is None:
            return {"call": False, "reroll": False, "bet": max_bet_index}
        if bet_index >= max_bet_index or bet_index == 109:
            return {"call": True}
        else:
            return {"call": False, "reroll": False, "bet": max_bet_index}



In [94]:
players = [SimpleAgent(), MaxAgent(), MaxAgent(), MaxAgent()]

In [98]:
simple_agent = SimpleAgent()
game = cmb.Game(num_players=4)

count = 0
for player in game.next_player():

    state = game.get_state(player)
    # print(state)

    game.turn(players[player].policy(state))
    count += 1
    if count > 10000:
        break

####################
new round
remaining dice per player {0: 5, 1: 5, 2: 5, 3: 5}
player 2 bet (4, 5)
player 0 called (4, 5) with a difference of 1 dice
####################
new round
remaining dice per player {0: 4, 1: 5, 2: 5, 3: 5}
player 1 bet (4, 5)
player 3 called (4, 5) with a difference of -1 dice
####################
new round
remaining dice per player {0: 4, 1: 4, 2: 5, 3: 5}
player 2 bet (4, 5)
player 0 called (4, 5) with a difference of 2 dice
####################
new round
remaining dice per player {0: 2, 1: 4, 2: 5, 3: 5}
player 1 bet (6, 3)
player 3 called (6, 3) with a difference of 4 dice
####################
new round
remaining dice per player {0: 2, 1: 4, 2: 5, 3: 1}
player 2 bet (4, 1)
player 0 bet (4, 2)
player 1 called (4, 2) with a difference of -2 dice
####################
new round
remaining dice per player {0: 0, 1: 4, 2: 5, 3: 1}
player 3 bet (4, 3)
player 2 called (4, 3) with a difference of 3 dice
####################
new round
remaining dice per player {0:

In [61]:
player

2